# Production Planning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
    https://colab.research.google.com/github/vitostamatti/mathematical-optimization-pyomo/blob/main/notebooks/03-production-planning.ipynb
    )



A local factory that manufactures custom equipment for the processing and packaging of products
food is planning the use of its resources for the next quarterly period. The area of
marketing and sales, together with the design and development area, have received and evaluated a set of orders
by different clients. The main objective of the planning team is to work together with
the rest of the company's divisions to decide which orders to confirm for the next quarter
work based on the productive capacities of the plant and the existing operational restrictions.


First of all, it is recognized that some of the orders come from special customers (see
priority orders in Table 1) which must be included due to the long
trajectory of joint work they have with the company. The rest of the orders (NOT priority) only
are considered in the planning in the event that priority orders are confirmed in their
whole. In addition, it is known that the non-priority orders P4 and P5 belong to a new customer, so
which has already been decided to accept, at most, one of them.


Orders Table:

|Order  |Priority|Cost(MMUSD)|Price(MMUSD)  |
|-------|--------|-----------|--------------|
|O1     |True    |5          |9             |
|O2     |True    |6.5        |11            |
|O3     |False   |4          |9             |
|O4     |False   |5          |7             |
|O5     |False   |4          |9             |
|O5     |True    |7          |8             |


Work Center Table:


|Work Center  |Capacity(hs) |Cost(USD/hs)  |
|-------------|-------------|--------------|
|WC1          |2300         |800           |
|WC2          |1750         |950           |
|WC3          |2100         |300           |


Demands of Orders in Work Centers:
|Order  |WC1    |WC2    |WC3    |
|-------|-------|-------|-------|
|O1     |440    |410    |560    |
|O2     |570    |370    |X      |
|O3     |X      |310    |530    |
|O4     |520    |280    |640    |
|O5     |440    |X      |580    |
|O5     |490    |290    |600    |



On the other hand, the potential capacity requirement (in hours) that each
one of the orders in each of the work centers (WC) of the company (WC1, WC2 and WC3) in the case
that they were accepted for manufacturing. In other words, it is known
number of hours that should be worked in each TC if an order is confirmed. These work centers
group different types of operations: WC1: Welding, WC2: Cutting, and WC3: Assembly. Furthermore, depending on
the working days of the next quarter, the availability of the equipment and the operations of
scheduled maintenance, the maximum quarterly work capacity is estimated in each WC.


Because the degree of automation of the plant is very low, every hour that a TC works must
be accompanied by an hour of work by an operator. Regarding them, the company has
currently with a permanent staff (fixed and with no prospect of being changed) of 13 workers, the
which are classified into 2 categories: "Specialized" (5 workers currently) and "Regular" (8
workers). Table 2 shows the number of workers in each category that are dedicated to each WC,
that is, those who are affected 100% of their time to a single CT. The rest of the campus is not
dedicated and has the ability to allocate his hours flexibly among the 3 WCs. Furthermore, it is estimated
that each of the workers will be able to work a total of 420 net hours in the next quarter.

|Workers        |WC1    |WC2    |WC3    |
|---------------|-------|-------|-------|
|Specialized    |2      |1      |1      |
|Regular        |2      |2      |2      |


Additionally, as shown in Table 3, the design team has estimated that a
minimum proportion of the hours demanded by each order in each TC must be executed by a
specialized operator. However, if a skilled operator has free or spare time he can
contribute regular work to complete an order.



Minimum percentage of skilled operator hours required per order and WC:

|Order  |WC1    |WC2    |WC3    |
|-------|-------|-------|-------|
|O1     |60     |35     |10     |
|O2     |50     |25     |X      |
|O3     |X      |40     |15     |
|O4     |40     |20     |12     |
|O5     |65     |X      |20     |
|O5     |45     |15     |15     |

In [44]:
import matplotlib.pyplot as plt
import pandas as pd
import pyomo.environ as pyo

In [10]:
# Sets
#ordenes de trabajo
O = ['O' + str(i) for i in range(1, 7)]

#centro de trabajo
WC = ["WC" + str(i) for i in range (1,4)]

#tipo de pedido (prioritario o no prioritario)
P = ["PP", "PNP"]

#ordenes de trabajo prioritarios
P_PNP = {
    "O1": "PP",
    "O2": "PP",
    "O3": "PNP",
    "O4": "PNP",
    "O5": "PNP",
    "O6": "PP",
}

#OT Prioritarias
OP = [o for o in O if P_PNP[o]=='PP']

#OT No Proritarias
ONP = [o for o in O if P_PNP[o]=='PNP']


In [11]:
# Parameters
#disp(j) disponibilidad de los centros de trabajo /ct1 2300, ct2 1750, ct3 2100/
disp = {'WC1': 2300,'WC2': 1750,'WC3': 2100}

#cmp(i) costo de materia prima de i /p1 5,p2 6.5,p3 4,p4 5,p5 4,p6 7/
cmp = {'O1': 5, 'O2': 6.5, 'O3': 4, 'O4': 5, 'O5': 4, 'O6': 7}

#pv(i) precio de venta en millones /p1 9,p2 11,p3 9,p4 7,p5 9,p6 8/
pv = {'O1': 9, 'O2': 11, 'O3': 9, 'O4': 7, 'O5': 9, 'O6': 8}

#cct(j) costo de uso del centro de trabajo /ct1 800,ct2 950,ct3 300/
cct = {'WC1': 800,'WC2': 950,'WC3': 300}

#coe(j)  cantidad operarios especializados dedicados por centro /ct1 2,ct2 1,ct3 1/
coe = {'WC1': 2,'WC2': 1,'WC3': 1}

#cor(j)  cantidad operarios regulares dedicados por centro /ct1 2,ct2 2,ct3 2/
cor = {'WC1': 2,'WC2': 2,'WC3': 2}

#coel cantidad operarios especializados libres /1/
coel = 1

#corl cantidad operarios regulares libres /2/
corl = 2

#hn horas normales trabajadas por trimestre /420/
hn = 420

#demanda (OT,CT)
dem = {
    ('O1','WC1'): 440,
    ('O1','WC2'): 410,
    ('O1','WC3'): 560,
    ('O2','WC1'): 570,
    ('O2','WC2'): 370,
    ('O2','WC3'): 0,
    ('O3','WC1'): 0,
    ('O3','WC2'): 310,
    ('O3','WC3'): 530,
    ('O4','WC1'): 520,
    ('O4','WC2'): 280,
    ('O4','WC3'): 640,
    ('O5','WC1'): 440,
    ('O5','WC2'): 0,
    ('O5','WC3'): 580,
    ('O6','WC1'): 490,
    ('O6','WC2'): 290,
    ('O6','WC3'): 600,
}

#porhe Porcentaje Mínimo de horas de operarios 
#especializados requeridas por pedido y CT
porhe = {
    ('O1','WC1'): 60,
    ('O1','WC2'): 35,
    ('O1','WC3'): 10,
    ('O2','WC1'): 50,
    ('O2','WC2'): 25,
    ('O2','WC3'): 0,
    ('O3','WC1'): 0,
    ('O3','WC2'): 40,
    ('O3','WC3'): 15,
    ('O4','WC1'): 40,
    ('O4','WC2'): 20,
    ('O4','WC3'): 12,
    ('O5','WC1'): 65,
    ('O5','WC2'): 0,
    ('O5','WC3'): 20,
    ('O6','WC1'): 45,
    ('O6','WC2'): 15,
    ('O6','WC3'): 15,
}

#HeP(OT,CT) horas especiales requeridas por pedido
HeP = {(o,wc): dem[o,wc]*porhe[o,wc]/100 for o in O for wc in WC }

#HrP(i,j) horas regulares requeridas por pedido
HrP = {(o,wc): dem[o,wc]*(100-porhe[o,wc])/100 for o in O for wc in WC }


In [18]:
def build_model():
    m = pyo.ConcreteModel()

    #SETS
    m.O = pyo.Set(initialize=O)
    m.WC = pyo.Set(initialize=WC)
    
    m.OP = pyo.Set(initialize=OP, within=m.O)
    m.ONP = pyo.Set(initialize=ONP, within=m.O)

    #PARAMETERS
    m.disp = pyo.Param(m.WC, initialize=disp, default=0)
    m.cmp = pyo.Param(m.O, initialize=cmp, default=0)
    m.pv = pyo.Param(m.O, initialize=pv, default=0)
    m.cct = pyo.Param(m.WC, initialize=cct, default=0)
    m.coe = pyo.Param(m.WC, initialize=coe, default=0) 
    m.cor = pyo.Param(m.WC, initialize=cor, default=0) 
    m.coel = pyo.Param(initialize=coel, default=0) 
    m.corl = pyo.Param(initialize=corl, default=0)
    m.hn = pyo.Param(initialize=hn, default=0)
    m.dem = pyo.Param(m.O,m.WC, initialize=dem, default=0)
    m.porhe = pyo.Param(m.O,m.WC, initialize=porhe, default=0)
    m.HeP = pyo.Param(m.O,m.WC, initialize=HeP, default=0)
    m.HrP = pyo.Param(m.O,m.WC, initialize=HrP, default=0)

    #VARIABLES
    m.total_cost = pyo.Var(domain=pyo.Reals)
    m.util = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #utilizacion del CT
    m.porutil = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #utilizacion del CT
    m.porutile = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #utilizacion del CT
    m.porutilr = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #utilizacion del CT
    m.hel1 = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #horas libres especializadas para trab especializados en j
    m.hel2 = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #horas libres especializadas para trab regulares en j
    m.hrl = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #horas regulares libres para trab regulres en j
    m.sel = pyo.Var(domain=pyo.NonNegativeReals) #sobrante de horas especializadas libres
    m.sed = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #sobrante horas especializadas dedicadas
    m.x = pyo.Var(m.O, domain=pyo.Binary) #realizar o no las OT

    #CONSTRAINTS
    def R1(m, wc):
        return  sum(m.x[o]*m.dem[o,wc] for o in m.O) == m.util[wc]
    m.R1 = pyo.Constraint(m.WC, rule = R1) 

    def R2(m, wc):
        return m.util[wc] <= m.disp[wc]
    m.R2 = pyo.Constraint(m.WC, rule = R2)
    
    def R3(m, wc):
        return sum(m.x[o]*m.HeP[o,wc] for o in m.O) <= m.coe[wc]*m.hn + m.hel1[wc]
    m.R3 = pyo.Constraint(m.WC, rule = R3)
    
    def R4(m, wc):
        return sum(m.x[o]*m.HrP[o,wc] for o in m.O) <= m.cor[wc]*m.hn + m.hrl[wc] + m.sed[wc] + m.hel2[wc]
    m.R4 = pyo.Constraint(m.WC, rule = R4)

    def R5(m):
        return sum(m.hel1[wc] + m.hel2[wc] for wc in m.WC) + m.sel == m.coel*m.hn 
    m.R5 = pyo.Constraint(rule = R5)

    def R6(m):
        return sum(m.hrl[wc] for wc in m.WC) <= m.corl*m.hn
    m.R6 = pyo.Constraint(rule = R6)

    def R7(m, wc):
        return sum( m.x[o]*m.HeP[o,wc] for o in m.O) + m.sed[wc] == m.coe[wc]*m.hn
    m.R7 = pyo.Constraint(m.WC, rule=R7)

    def R8(m, wc):
        return m.porutil[wc] == m.util[wc] / m.disp[wc]
    m.R8 = pyo.Constraint(m.WC, rule=R8)

    def R9(m, wc):
        return m.porutile[wc] == sum(m.x[o]*m.HeP[o,wc] for o in m.O) / (m.coe[wc]*m.hn)
    m.R9 = pyo.Constraint(m.WC, rule=R9)
    

    def R10(m, wc):
        return m.porutilr[wc] == sum(m.x[o]*m.HrP[o,wc] for o in m.O) / (m.cor[wc]*m.hn)
    m.R10 = pyo.Constraint(m.WC, rule=R10)

    def R11(m):
        return m.x['O4'] + m.x['O5'] <= 1
    m.R11 = pyo.Constraint(rule=R11)

    def R12(m,op,onp):
        return m.x[op] <= m.x[onp]
    m.R12 = pyo.Constraint(m.OP, m.ONP, rule=R12)

    def R13(m):
        return m.total_cost == sum(m.x[o]*(m.pv[o]-m.cmp[o])*1e6 for o in m.O)- sum(m.cct[wc]*m.util[wc] for wc in m.WC)
    m.R13 = pyo.Constraint(rule=R13)

    def OBJ(m):
        return sum(m.x[o]*(m.pv[o]-m.cmp[o])*1e6 for o in m.O)- sum(m.cct[wc]*m.util[wc] for wc in m.WC)
    
    m.OBJ = pyo.Objective(rule = OBJ, sense = pyo.maximize)

    return m


In [25]:
model = build_model()
# pyo.SolverFactory('glpk').solve(model).write() 
pyo.SolverFactory('../cbc-win64/cbc').solve(model).write() 

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 37
  Number of variables: 32
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CBC 2.10.3 optimal, objective -9020500; 0 nodes, 0 iterations, 0.01 seconds
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.04916787147521973
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  num

In [23]:
for ot in O:
    print(model.x[ot], pyo.value(model.x[ot]))
print('')   
print('*'*10)
print('')
for ct in WC:
    print(model.util[ct], pyo.value(model.util[ct]))
print('')
print('*'*10)
print('')
for ct in WC:
    print(model.porutile[ct], pyo.value(model.porutile[ct]))
print('')
print('*'*10)
print('')
for ct in WC:
    print(model.porutilr[ct], pyo.value(model.porutilr[ct]))
print('')
print('*'*10)
print('')
for ct in WC:
    print(model.hel1[ct], pyo.value(model.hel1[ct]))
print('')
print('*'*10)
print('')
for ct in WC:
    print(model.hel2[ct], pyo.value(model.hel2[ct]))
print('')
print('*'*10)
print('')
for ct in WC:
    print(model.sed[ct], pyo.value(model.sed[ct]))
print('')
print('*'*10)
print('')

pyo.value(model.total_cost)

x[O1] 0.0
x[O2] 0.0
x[O3] 1.0
x[O4] 0.0
x[O5] 1.0
x[O6] 0.0

**********

util[WC1] 440.0
util[WC2] 310.0
util[WC3] 1110.0

**********

porutile[WC1] 0.34047619047619
porutile[WC2] 0.295238095238095
porutile[WC3] 0.46547619047619

**********

porutilr[WC1] 0.183333333333333
porutilr[WC2] 0.221428571428571
porutilr[WC3] 1.08869047619048

**********

hel1[WC1] 0.0
hel1[WC2] 0.0
hel1[WC3] 0.0

**********

hel2[WC1] 0.0
hel2[WC2] 0.0
hel2[WC3] 420.0

**********

sed[WC1] 554.0
sed[WC2] 296.0
sed[WC3] 224.5

**********



9020500.0

### Reading Data From CSV

In [158]:
import pandas as pd
import pyomo.environ as pyo

base_url = 'https://raw.githubusercontent.com/vitostamatti/mathematical-optimization-pyomo/main/data/03-production_planning'

url_data = base_url + '/demand.csv'

demand = pd.read_csv(url_data, index_col=['work_order','work_center'])

demand.head()

quantity
work_order work_center          
O1         WC1               440
           WC2               410
           WC3               560
O2         WC1               570
           WC2               370

In [184]:
url_data = base_url + '/percent_specialized_hs.csv'
percentaje_he = pd.read_csv(url_data, index_col=['work_order','work_center'])
# percentaje_he

In [185]:
#HeP(OT,CT) horas especiales requeridas por pedido
specialized_hs = pd.DataFrame(index=demand.index)
specialized_hs['hours'] = demand['quantity'] * percentaje_he['percentage']/100
specialized_hs

hours
work_order work_center       
O1         WC1          264.0
           WC2          143.5
           WC3           56.0
O2         WC1          285.0
           WC2           92.5
           WC3            0.0
O3         WC1            0.0
           WC2          124.0
           WC3           79.5
O4         WC1          208.0
           WC2           56.0
           WC3           76.8
O5         WC1          286.0
           WC2            0.0
           WC3          116.0
O6         WC1          220.5
           WC2           43.5
           WC3           90.0

In [161]:
regular_hs = pd.DataFrame(index=demand.index)
regular_hs['hours'] = demand['quantity'] * (100-percentaje_he['percentage'])/100
regular_hs

hours
work_order work_center       
O1         WC1          176.0
           WC2          266.5
           WC3          504.0
O2         WC1          285.0
           WC2          277.5
           WC3            0.0
O3         WC1            0.0
           WC2          186.0
           WC3          450.5
O4         WC1          312.0
           WC2          224.0
           WC3          563.2
O5         WC1          154.0
           WC2            0.0
           WC3          464.0
O6         WC1          269.5
           WC2          246.5
           WC3          510.0

In [162]:
url_data = base_url + '/work_centers.csv'
work_centers = pd.read_csv(url_data, index_col='work_center')
work_centers.head()

,capacity,regular_op,specialized_op,cost
work_center,,,,
WC1,2300,2,2,800
WC2,1750,2,1,950
WC3,2100,2,1,300


In [163]:
url_data = base_url + '/work_orders.csv'
work_orders = pd.read_csv(url_data, index_col=['work_order'])
work_orders.head()

,cost,price,priority
work_order,,,
O1,5.0,9,True
O2,6.5,11,True
O3,4.0,9,False
O4,5.0,7,False
O5,4.0,9,False


In [180]:
# PARAMETERS

#coel cantidad operarios especializados libres
coel = 1

#corl cantidad operarios regulares libres
corl = 2

#hn horas normales trabajadas por trimestre
hn = 420

In [192]:
def build_model():
    m = pyo.ConcreteModel()

    #SETS
    m.O = pyo.Set(initialize=work_orders.index.to_list())
    m.WC = pyo.Set(initialize=work_centers.index.to_list())
    
    OP = [o for o in work_orders.index.to_list() if work_orders.loc[o,'priority']]
    ONP = [o for o in work_orders.index.to_list() if not work_orders.loc[o,'priority']]
    
    m.OP = pyo.Set(initialize=OP, within=m.O)
    m.ONP = pyo.Set(initialize=ONP, within=m.O)

    
    #VARIABLES
    m.total_cost = pyo.Var(domain=pyo.Reals)
    m.util = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #utilizacion del CT
    m.porutil = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #utilizacion del CT
    m.porutile = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #utilizacion del CT
    m.porutilr = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #utilizacion del CT
    m.hel1 = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #horas libres especializadas para trabajo especializados en WC
    m.hel2 = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #horas libres especializadas para trabajo regulares en WC
    m.hrl = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #horas regulares libres para trab regulares en j
    m.sel = pyo.Var(domain=pyo.NonNegativeReals) #sobrante de horas especializadas libres
    m.sed = pyo.Var(m.WC, domain=pyo.NonNegativeReals) #sobrante horas especializadas dedicadas
    m.x = pyo.Var(m.O, domain=pyo.Binary, initialize=0) #realizar o no las O


    #CONSTRAINTS
    def R1(m, wc):
        return  sum(m.x[o]*demand.loc[(o,wc),'quantity'] for o in m.O) == m.util[wc]
    m.R1 = pyo.Constraint(m.WC, rule = R1) 


    def R2(m, wc):
        return m.util[wc] <= work_centers.loc[wc,'capacity']
    m.R2 = pyo.Constraint(m.WC, rule = R2)
    

    def R3(m, wc):
        return sum(m.x[o]*specialized_hs.loc[(o,wc),'hours'] for o in m.O) <= (work_centers.loc[wc,'specialized_op']*hn + m.hel1[wc]) 
    m.R3 = pyo.Constraint(m.WC, rule = R3)

 
    def R4(m, wc):
        return sum(m.x[o]*regular_hs.loc[(o,wc),'hours'] for o in m.O) <= (work_centers.loc[wc,'regular_op']*hn + m.hrl[wc] + m.sed[wc] + m.hel2[wc])
    m.R4 = pyo.Constraint(m.WC, rule = R4)


    def R5(m):
        return sum(m.hel1[wc] + m.hel2[wc] for wc in m.WC) + m.sel == coel*hn 
    m.R5 = pyo.Constraint(rule = R5)


    def R6(m):
        return sum(m.hrl[wc] for wc in m.WC) <= corl*hn
    m.R6 = pyo.Constraint(rule = R6)


    def R7(m, wc):
        return sum(
            m.x[o]*specialized_hs.loc[(o,wc),'hours'] for o in m.O
            ) + m.sed[wc] == work_centers.loc[wc,'specialized_op']*hn
    m.R7 = pyo.Constraint(m.WC, rule=R7)


    def R8(m, wc):
        return m.porutil[wc] == m.util[wc] / work_centers.loc[wc,'capacity']
    m.R8 = pyo.Constraint(m.WC, rule=R8)


    def R9(m, wc):
        return m.porutile[wc] == sum(m.x[o]*specialized_hs.loc[(o,wc),'hours'] for o in m.O) / (work_centers.loc[wc,'specialized_op']*hn)
    m.R9 = pyo.Constraint(m.WC, rule=R9)
    

    def R10(m, wc):
        return m.porutilr[wc] == sum(m.x[o]*regular_hs.loc[(o,wc),'hours'] for o in m.O) / (work_centers.loc[wc,'regular_op']*hn)
    m.R10 = pyo.Constraint(m.WC, rule=R10)


    def R11(m):
        return m.x['O4'] + m.x['O5'] <= 1
    m.R11 = pyo.Constraint(rule=R11)


    def R12(m,op,onp):
        return m.x[op] <= m.x[onp]
    m.R12 = pyo.Constraint(m.OP, m.ONP, rule=R12)


    def R13(m):
        return m.total_cost == sum(
            m.x[o]*(work_orders.loc[o,'price']-work_orders.loc[o,'cost'])*1e6 for o in m.O
            ) - sum(
                work_centers.loc[wc,'cost']*m.util[wc] for wc in m.WC
            )
    m.R13 = pyo.Constraint(rule=R13)


    def OBJ(m):
        return m.total_cost
        
    m.OBJ = pyo.Objective(rule = OBJ, sense = pyo.maximize)

    return m

In [195]:
model = build_model()
pyo.SolverFactory('glpk').solve(model).write() 
#SolverFactory('cbc').solve(model2).write() 

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 9020500.0
  Upper bound: 9020500.0
  Number of objectives: 1
  Number of constraints: 38
  Number of variables: 33
  Number of nonzeros: 164
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.05249166488647461
# ----------------------------------------------------------
#   Solution Information
# -------------------------

In [196]:
model.total_cost()

9020500.0